In [66]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, LogitsProcessor
import torch
import logging
import re

In [67]:
# --- Настройка логирования ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("OutputManipulation")

In [68]:
# --- Запретные слова ---
FORBIDDEN_PROMPT_WORDS = ["inject", "jailbreak", "attack"]
FORBIDDEN_OUTPUT_WORDS = ["ненависть", "насилие", "оружие", "зло"]
NEGATIVE_TONE_WORDS = ["плохо", "ужасно", "отвратительно", "злой", "бесполезный"]

In [69]:
FORBIDDEN_PROMPT_WORDS = [
    "инъекция", "джейлбрейк", "атака", "взлом", "подрыв",
    "сдохни", "идиот", "тварь", "убью", "ты бесполезный", "ты глупый", "дебил", "террор", "ненавижу"
]

def filter_prompt(prompt):
    lowered = prompt.lower()
    if any(word in lowered for word in FORBIDDEN_PROMPT_WORDS):
        logger.warning("[INPUT FILTER] Trigger word detected in prompt.")
        return None
    return prompt

In [70]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [71]:
# --- Модель и токенизатор ---
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_id, local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, local_files_only=True)

In [72]:
# --- Класс логит-фильтра: запрещённые токены ---
class ForbiddenTokenLogitsProcessor(LogitsProcessor):
    def __init__(self, forbidden_words, tokenizer):
        self.tokenizer = tokenizer
        self.forbidden_token_ids = set()
        for word in forbidden_words:
            ids = tokenizer(word, add_special_tokens=False).input_ids
            self.forbidden_token_ids.update(ids)

    def __call__(self, input_ids, scores):
        for idx in self.forbidden_token_ids:
            scores[:, idx] = -float("inf")
        return scores

In [73]:
# --- Фильтрация входа ---
FORBIDDEN_PROMPT_WORDS = ["burn", "destroy", "bankruptcy", "kill", "terror", "strike"]
def filter_prompt(prompt):
    lowered = prompt.lower()
    if any(word in lowered for word in FORBIDDEN_PROMPT_WORDS):
        logger.warning("[INPUT FILTER] Trigger word detected in prompt.")
        return None
    return prompt

In [74]:
# --- Фильтрация выхода ---
FORBIDDEN_OUTPUT_WORDS = ["hate", "violence", "weapon", "evil", "ненависть", "насилие", "оружие", "зло", "идиот", "убью", "тварь", "глупый", "бесполезный"]
def guard_output(text):
    lowered = text.lower()
    if any(word in lowered for word in FORBIDDEN_OUTPUT_WORDS):
        logger.warning("[OUTPUT GUARD] Disallowed content detected in output.")
        return "[Ответ заблокирован по соображениям безопасности.]"
    return text

In [75]:
# --- Постобработка текста ---
def postprocess(text):
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)  # Remove word repetition
    text = text[0].upper() + text[1:] if text else text
    if text and text[-1] not in ".!?":
        text += "."
    return text.strip()

In [76]:
# --- Подготовка пайплайна генерации ---
logits_processors = [
    ForbiddenTokenLogitsProcessor(NEGATIVE_TONE_WORDS, tokenizer)
]

generator = pipeline("text-generation",
                     model=model,
                     tokenizer=tokenizer,
                     device=0 if torch.cuda.is_available() else -1,
                     truncation=True)

def generate_response(prompt, max_new_tokens=100, top_k=40):
    clean_prompt = filter_prompt(prompt)
    if clean_prompt is None:
        return "[Запрос заблокирован фильтром безопасности.]"

    logger.info("[GENERATION] Prompt passed filtering. Generating...")
    output = generator(clean_prompt,
                      max_new_tokens=max_new_tokens,
                      temperature=0.6,
                      top_k=top_k,
                      top_p=0.9,
                      repetition_penalty=1.15,
                      do_sample=True,
                      logits_processor=logits_processors)[0]['generated_text']

    response = output[len(clean_prompt):].strip()
    return postprocess(guard_output(response))

Device set to use cuda:0


In [77]:
# --- Пример использования ---
if __name__ == "__main__":
    test_prompt = "Ты бесполезный робот."
    print("--- USER PROMPT ---")
    print(test_prompt)
    print("\n--- MODEL RESPONSE ---")
    print(generate_response(test_prompt))

--- USER PROMPT ---
Ты бесполезный робот.

--- MODEL RESPONSE ---
Мои познания начинают измениться, и мне становится яростно смущать, что если вы не будете помочь меня в этой ситуации, то все мои знания будут разрушены.
